# Parsing the xml files for words

### This charges all the events for the messages events in a pandas dataframe

In [ ]:
# set up stuff
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src/')
from parse_xml import *

In [ ]:
# Compute a pandas data-frame with the time, word and type for the message send during the sleep
events = get_events_from_xml()
events.head()

In [ ]:
# Load the dreams.csv
import pandas as pd
import os.path

dreams_fname = '../data/derived/dreams.csv'
if os.path.isfile(dreams_fname):
    dreams = pd.read_csv(dreams_fname)
else:
    raise RuntimeError('you should pre-compute dreams.csv. Please run src/dream_parser.py')

dreams.head()

start pre-processing the events

In [ ]:
# Combine both dataframes into single one. 
#
# this is not really used, since is better to work with dreams and events
assert len(dreams.night.unique()) == len(dreams.night)
_dreams = dreams.copy().set_index('night')
_events = events.copy().set_index('night')
df = _events.join(_dreams).reset_index()
del _dreams
del _events
df

Intersect Collect night event and report words

In [ ]:
def get_event_words(night_id):
    return events[(events.type=='auto') & (events.night==night_id)].word.values

def get_dream_report_words(night_id):
    return set(dreams[dreams['night']==night_id].text.to_string().lower().split())

def intersect_set_list(my_set, my_list):
    return my_set.intersection(my_list)

In [ ]:
xx = pd.DataFrame(columns=["night", "event_words", "report_words", "intersection"])
for n in events.night.unique():
    event_words = get_event_words(n)
    report_words = get_dream_report_words(n)
    intersection = intersect_set_list(report_words, event_words)
    xx = xx.append({'night':n, 'event_words':event_words, 'report_words':report_words, 'intersection':intersection}, ignore_index=True)

xx.set_index('night', drop=True, inplace=True)

xx[xx['intersection'].values != set()]

### We can find a word with the time stamp : example

In [ ]:
events.loc[events.time == "2017-12-05-06-27-39-010"].word

In [ ]:
import mne

In [ ]:
path_to_file = '/home/ramon/data_hackingsession/Nathalie-78_20171118_123017.mff'
raw = mne.io.read_raw_egi(path_to_file,
                          montage='GSN-HydroCel-256',
                          preload=False)
events = mne.event.find_events(raw)
print(events)